In [1]:
%reload_ext autoreload
%autoreload 2

import os
import json
import re

from openai import OpenAI
from dotenv import load_dotenv

load_dotenv("OpenAI.env")

client = OpenAI()

DIR = "output"

In [2]:
#Images_Intro  creation
from typing import List, Any, Optional

from ImageAgent import Slide, ImagePromptAgent, OpenAIAgentConfig

def process_intro_videos(image_intro) -> None:

    agent = ImagePromptAgent(client, OpenAIAgentConfig(
                                        model="gpt-5-nano",      # текстова модель для створення промта
                                        image_model="gpt-image-1" # модель для рендеру зображення (якщо потрібно)
                                        ))

    title = COURSE_DATA.get('course_name', '') + ' ' + image_intro
    bullets = COURSE_DATA.get('description', '')
    out_dir = COURSE_DATA.get('path', 'Untitled')
    
    print(out_dir)
    print(f"Creating intro content: {title}")
    print(f"Description: {bullets}")

            
    prompt = agent.generate_prompt(Slide(title=title, bullets= bullets))
    # print(prompt)
    #Image generation
    save_path = f"{image_intro}.png"
    agent.render_image(prompt, size="1536x1024", out_dir=out_dir, save_path=save_path)

In [3]:
#Images content creation
from typing import List, Any, Optional

from ExtractSlidesContentAgent import LectureSlidesAgent
from ImageAgent import Slide, ImagePromptAgent, OpenAIAgentConfig

def process_module_images(
    module: dict, 
    items: List[dict], 
    item_index: Optional[int] = None,
    slides_list: Optional[List] = [],
    ) -> None:

    agent = ImagePromptAgent(client, OpenAIAgentConfig(
                                        model="gpt-5-nano",      # текстова модель для створення промта
                                        image_model="gpt-image-1" # модель для рендеру зображення (якщо потрібно)
                                        ))
    
    if not items:
        print("  No videos found.")
        return
    
    if item_index is not None: #One Video
        if 1 <= item_index <= len(items):
            it = items[item_index - 1]
            print(f"Creating images for video content: {it.get('title', 'Untitled')}")
            
            json_path = it.get('path', 'Untitled')
            out_dir = os.path.join(json_path, "image_out")
            content_agent = LectureSlidesAgent(os.path.join(json_path, "content.json") )
            print(os.path.join(json_path, "content.json"))
            
            #slice for rebuild!!!!
            # print("Slides list", slides_list)
            if slides_list == []:
                s = [i for i in range(2, content_agent.slides_count()+1)] 
            else:
                s = slides_list

            print("Creating images for slides ", s)
            
            for idx in s:
                title = content_agent.get_title(idx)
                bullets = content_agent.get_content(idx)
            
                print("=" * 80 )
                print(f"Slide {idx}")
                print(f"Title: {title}")
                print("Bullets:")
                if bullets:
                    for b in bullets:
                        print(f" - {b}")
                else:
                    print(" - (no bullets found)")
                #prompt for image generation
                prompt = agent.generate_prompt(Slide(title=title, bullets= bullets))
                # print(prompt)
                #Image generation
                save_path = f"Slide_{idx}.png"
                agent.render_image(prompt, size="1536x1024", out_dir=out_dir, save_path=save_path)

            
    else: #Videos for whole module
        for i, it in enumerate(items, 1):
            print(f"Creating video content: {it.get('title', 'Untitled')}")
            
            json_path = it.get('path', 'Untitled')
            out_dir = os.path.join(json_path, "image_out")
            content_agent = LectureSlidesAgent(os.path.join(json_path, "content.json") )
            print(os.path.join(json_path, "content.json"))
            
            s = [i for i in range(2, content_agent.slides_count()+1)] 
            print("Creating images for slides ", s)
            
            for idx in s:
                title = content_agent.get_title(idx)
                bullets = content_agent.get_content(idx)
            
                print("=" * 80 )
                print(f"Slide {idx}")
                print(f"Title: {title}")
                print("Bullets:")
                if bullets:
                    for b in bullets:
                        print(f" - {b}")
                else:
                    print(" - (no bullets found)")
                #prompt for image generation
                prompt = agent.generate_prompt(Slide(title=title, bullets= bullets))
                # print(prompt)
                #Image generation
                save_path = f"Slide_{idx}.png"
                agent.render_image(prompt, size="1536x1024", out_dir=out_dir, save_path=save_path)

In [4]:
#Audio content creation
from typing import List, Any, Optional

from VoiceAgent import PresentationVoiceAgent
from ExtractSlidesContentAgent import LectureSlidesAgent

def process_module_audio(
    module: dict, 
    items: List[dict], 
    item_index: Optional[int] = None,
    slides_list: Optional[List] = [],
    ) -> None:

    instructions = (
        "Read the text with a tone that is clear, professional, high energy and confident, but not monotonous. "
        "Add slight warmth and emphasis so learners stay interested. "
        "Use natural pauses and vary your sentence rhythm to keep the narration dynamic. "
        "Avoid sounding like you’re just reading; instead, guide the learner as if you are teaching them step by step. "
        "Keep the delivery professional but conversational, as if you’re explaining important ideas to someone who wants to understand, not just memorize."
    )

    if not items:
        print("  No videos found.")
        return
    
    if item_index is not None: #One Video
        if 1 <= item_index <= len(items):
            it = items[item_index - 1]
            print(f"Creating audio for video content: {it.get('title', 'Untitled')}")
            
            json_path = it.get('path', 'Untitled')
            out_dir = os.path.join(json_path, "audio_out")
    
            agent = PresentationVoiceAgent(client,
                                           model="tts-1",
                                           voice="nova",
                                           out_dir=out_dir,
                                           instructions=instructions,
                                           verbose=False)

            
            content_agent = LectureSlidesAgent(os.path.join(json_path, "content.json") )
            print(os.path.join(json_path, "content.json"))
            
            #slice for rebuild!!!!
            # print("Slides list", slides_list)

            if slides_list == []:
                s = [i for i in range(1, content_agent.slides_count()+1)] 
            else:
                s = slides_list

            print("Creating audio for slides ", s)
            
            for idx in s:
                notes = content_agent.get_notes(idx)            
                # print("=" * 80 )
                # print(f"Slide {idx}")
                # print(f"Title: {title}")
                # print(f"Notes: {notes}")
                # print("Bullets:")
                # if bullets:
                #     for b in bullets:
                #         print(f" - {b}")
                # else:
                #     print(" - (no bullets found)")
                #prompt for audio generation
                agent.process(idx, notes)

            
    else: #Audio for whole module
        for i, it in enumerate(items, 1):
            print(f"Creating video content: {it.get('title', 'Untitled')}")
            
            json_path = it.get('path', 'Untitled')
            out_dir = os.path.join(json_path, "audio_out")
    
            agent = PresentationVoiceAgent(client,
                                           model="tts-1",
                                           voice="nova",
                                           out_dir=out_dir,
                                           instructions=instructions,
                                           verbose=False)

            
            content_agent = LectureSlidesAgent(os.path.join(json_path, "content.json") )
            print(os.path.join(json_path, "content.json"))
            
            #slice for rebuild!!!!
            # print("Slides list", slides_list)

            s = [i for i in range(1, content_agent.slides_count()+1)] 
            print("Creating audio for slides ", s)
            
            for idx in s:
                notes = content_agent.get_notes(idx)            
                # print("=" * 80 )
                # print(f"Slide {idx}")
                # print(f"Title: {title}")
                # print(f"Notes: {notes}")
                # print("Bullets:")
                # if bullets:
                #     for b in bullets:
                #         print(f" - {b}")
                # else:
                #     print(" - (no bullets found)")
                
                # prompt for audio generation
                agent.process(idx, notes)


In [5]:
import json
import os
from typing import List, Any, Optional

# ==============================================================
# НАЛАШТУВАННЯ
# ==============================================================
JSON_FILE = "AML_Uncovered_curriculum_with_paths.json"

# ==============================================================
# 1. ЗАВАНТАЖЕННЯ ДАНИХ
# ==============================================================
json_file_with_path = os.path.join(DIR, JSON_FILE)
with open(json_file_with_path, "r", encoding="utf-8") as f:
    COURSE_DATA = json.load(f)

# ==============================================================
# ГЛОБАЛЬНІ КОНСТАНТИ
# ==============================================================
MODULE_CONTENT_KEYS = ["videos", "readings", "case_studies", "quiz"]
EXAM_CONTENT_KEYS = ["case_studies", "quiz"]

# ==============================================================
# 4. МЕНЕДЖЕР ДЛЯ МОДУЛІВ
# ==============================================================
def content_manager(
    module_num: int,
    content_key: str,
    item_index: Optional[int] = None,
    slides_list: Optional[List] = [],
    content_type: Optional[List] = ['Image', 'Audio', 'pptx', 'Video'],  
) -> None:

    
    if content_key not in MODULE_CONTENT_KEYS:
        print(f"ERROR: Invalid content key for module: '{content_key}'")
        return
    modules = COURSE_DATA.get("modules", [])
    if module_num < 1 or module_num > len(modules):
        print(f"ERROR: Module {module_num} not found. Available: 1-{len(modules)}")
        return
    module = modules[module_num - 1]
    items = module.get(content_key, [])
    mod_path = module.get("path", "N/A")
    print(f"\nMODULE {module_num} — {content_key.upper()}")
    print("—" * 50)
    
    if content_key == "videos":
        if 'Image' in content_type: 
            process_module_images(module, items, item_index, slides_list)
        if 'Audio' in content_type: 
            process_module_audio(module, items, item_index, slides_list)


# ==============================================================
# 6. ГОЛОВНА ФУНКЦІЯ
# ==============================================================
def display_content(
    module_num: Optional[int] = None, #номер модуля
    content_key: Optional[str] = None, #ключ контенту (Video)
    item_index: Optional[int] = None, #порядковий номер контенту. 
    exam: bool = False, #екзамен
    slides_list: Optional[List] = [], # список складів по яким потрібно зробити зміну []-всі слайди
    image_intro: Optional[str] = None, #  Inrtro or Outro створення картинки початковий чи кінцевий 
    content_type: Optional[List] = ['Image', 'Audio', 'pptx', 'Video'], #що ми створюємо   
    
) -> None:
    if exam:
        if content_key is not None:
            exam_content_manager(content_key, item_index)
        else:
            print(f"COURSE: {COURSE_DATA.get('course_name')} — FINAL EXAM")
            print("=" * 80)
            for key in EXAM_CONTENT_KEYS:
                pass
                # exam_content_manager(key)
    elif image_intro is not None:
        process_intro_videos(image_intro)        
    else:
        if module_num is not None and content_key is not None:
            content_manager(module_num, content_key, item_index=item_index, slides_list=slides_list, content_type=content_type)
        elif module_num is None and content_key is None:
            modules = COURSE_DATA.get("modules", [])
            print(f"COURSE: {COURSE_DATA.get('course_name')} | {len(modules)} MODULES")
            print("=" * 80)
            for mod_idx in range(1, len(modules) + 1):
                for key in MODULE_CONTENT_KEYS:
                    content_manager(mod_idx, key, item_index=item_index, content_type=content_type)
                print()
        else:
            print("ERROR: Invalid arguments.")
            print("   display_content(module_num, content_key, item_index) → one item")
            print("   display_content(module_num, content_key)             → all in key")
            print("   display_content()                                   → all modules")
            print("   display_content(exam=True, ...)                     → exam only")


# ==============================================================
# ДЕМОНСТРАЦІЯ
# ==============================================================
if __name__ == "__main__":
    import time

    start = time.perf_counter()

    
    # print("DEMO 1: Усі модулі")
    # print("=" * 80)
    # display_content()

    # print("\n" + "=" * 80)
    # print("DEMO 2: Модуль 1 — Video 1")
    # print("=" * 80)
    # display_content(module_num = 1, content_key = "videos", item_index = 1, slides_list = [3,], content_type = ['Image'] )

    # print("\n" + "=" * 80)
    # print("DEMO 3: Модуль 1 — Video 1")
    # print("=" * 80)
    # display_content(module_num = 1, content_key = "videos", item_index = 1)

    # print("\n" + "=" * 80)
    # print("DEMO 4: Intro & Outro")
    # print("=" * 80)
    # display_content(image_intro = "Intro")
    # display_content(image_intro = "Outro")
    
    print("\n" + "=" * 80)
    print("DEMO 5: Модуль 1 — Audio 1")
    print("=" * 80)
    display_content(module_num = 1, content_key = "videos", item_index = 1, slides_list = [3,], content_type = ['Audio'] )
    
    # print("\n" + "=" * 80)
    # print("DEMO 6: Модуль 1 - аудіо для всього модуля")
    # print("=" * 80)
    # display_content(module_num = 1, content_key = "videos", content_type = ['Audio'] )

    # print("\n" + "=" * 80)
    # print("DEMO 7: створення аудіо для всього курсу")
    # print("=" * 80)
    # display_content(content_type = ['Audio'] )

    

    elapsed = time.perf_counter() - start
    minutes = int(elapsed // 60)
    seconds = elapsed % 60
    
    print(f"Elapsed time: {minutes} min {seconds:.2f} sec")


DEMO 5: Модуль 1 — Audio 1

MODULE 1 — VIDEOS
——————————————————————————————————————————————————
Creating audio for video content: Foundations of AML: Key Concepts and Definitions
output/AML_Uncovered/module1/video1/content.json
Creating audio for slides  [3]
Elapsed time: 0 min 23.34 sec
